In [26]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [27]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/rf_regressor.csv')

In [28]:
### Season to test results

N = 2021

In [29]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop([
        'season', 'round',
        'podium','driver_points_from_race'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from_race.values)

In [30]:
X_train

,driver_standings_pos,constructor_standings_pos,starting_grid,driver_points_last_3_races,driver_adrian_sutil,driver_alexander_albon,driver_alexander_rossi,driver_andre_lotterer,driver_antonio_giovinazzi,driver_brendon_hartley,...,circuit_id_rodriguez,circuit_id_sepang,circuit_id_shanghai,circuit_id_silverstone,circuit_id_sochi,circuit_id_spa,circuit_id_suzuka,circuit_id_villeneuve,circuit_id_yas_marina,circuit_id_zandvoort
0,-1.595994,-1.697284,-1.294514,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,-0.230289,0.0
1,-1.595994,-1.697284,-1.122576,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,-0.230289,0.0
2,-1.595994,-1.697284,-0.090952,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,-0.230289,0.0
3,-1.595994,-1.697284,-0.950639,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,-0.230289,0.0
4,-1.595994,-1.697284,0.768734,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,-0.230289,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,1.067971,0.856971,0.596797,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,8.352798,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,4.342363,0.0
2756,1.694786,1.495535,1.284546,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,4.342363,0.0
2757,1.538082,1.176253,1.628421,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,4.342363,0.0
2758,2.008193,1.176253,1.112609,-0.753403,-0.083257,-0.116567,-0.042601,-0.019038,-0.119720,-0.09167,...,-0.191871,-0.174985,-0.21227,-0.248069,-0.232895,-0.232895,-0.214128,-0.21227,4.342363,0.0


In [31]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop([
            'season', 'round',
            'podium','driver_points_from_race'], axis=1)
        y_test = test.driver_points_from_race

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        points += list(prediction_df.predicted_points.values)
        winners += list(prediction_df.predicted_winner.values)

        # Retrain model
        X_train = pd.concat([X_train, X_test])
        y_train = np.append(y_train, y_test)

        model.fit(X_train, y_train)
        

    return points, winners

In [36]:
parameters.head()

,model,criterion,max_features,max_depth,score
0,random_forest_regressor,squared_error,1.0,13,0.590909
1,random_forest_regressor,squared_error,1.0,21,0.590909
2,random_forest_regressor,squared_error,1.0,47,0.590909
3,random_forest_regressor,squared_error,0.8,27,0.590909
4,random_forest_regressor,squared_error,1.0,41,0.590909


In [33]:
params = parameters.iloc[1]
criterion = params.criterion
max_features = params.max_features	
max_depth = params.max_depth

params

model           random_forest_regressor
criterion                 squared_error
max_features                        1.0
max_depth                            21
score                          0.590909
Name: 1, dtype: object

In [34]:
# Random Forest Regressor

test = processed.copy()
test = test[['season', 'round', 'driver', 'starting_grid', 'podium', 'driver_points_from_race', 'q_delta']].query('season ==@N')

model_params = (criterion, max_features, max_depth)
model = RandomForestRegressor(criterion=criterion, max_features=max_features, max_depth=max_depth)
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [35]:
test.query('predicted_winner == 1')

,season,round,driver,starting_grid,podium,driver_points_from_race,q_delta,predicted_points,predicted_winner
2760,2021,1,lewis_hamilton,2,1,25.0,0.388,18.554738,1
2781,2021,2,lewis_hamilton,1,2,19.0,0.000,22.409286,1
2799,2021,3,lewis_hamilton,2,1,25.0,0.007,23.476429,1
2819,2021,4,lewis_hamilton,1,1,25.0,0.000,23.920000,1
2839,2021,5,max_verstappen,2,1,25.0,0.230,20.562222,1
2871,2021,6,lewis_hamilton,2,15,0.0,0.000,18.732500,1
2876,2021,7,max_verstappen,1,1,26.0,0.000,18.045511,1
2894,2021,8,max_verstappen,1,1,25.0,0.000,19.435921,1
2914,2021,9,max_verstappen,1,1,26.0,0.000,21.690000,1
2934,2021,10,lewis_hamilton,2,1,27.0,0.000,21.364429,1
